In [8]:
training_data=[
    ['green',3,'mango'],
    ['yellow',3,'mango'],
    ['red',1,'grape'],
    ['red',1,'grape'],
    ['yellow',3,'lemon']
]
#column labels
#used to print the tree.
header=["color","diameter","label"]
def unique_vals(rows,col):
    return set([row[col] for row in rows])

def class_counts(rows):
    counts={}
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
            
        counts[label] += 1
    return counts

def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

class Question:
    
    def __init__(self,column,value):
        self.column=column
        self.value=value
        
    def match(self,example):
        val=example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    def __repr__(self):
        condition="=="
        if is_numeric(self.value):
            condition = ">="
        return "is %s %s %s?" %(
           header[self.column],condition,str(self.value))
def partition(rows,question):
    true_rows,false_rows=[], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows
def gini(rows):
    counts= class_counts(rows)
    impurity=1
    for lbl in counts:
        prob_of_lbl= counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return  impurity

def info_gain(left,right,current_uncertainty):
    p=float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)
def find_best_split(rows):
    best_gain=0
    best_question = None
    current_uncertainty=gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values= set([row[col] for row in rows ])
        for val in values:
            question =Question(col,val)
            true_rows,false_rows= partition(rows,question)
            
            if len(true_rows) == 0 or len(false_rows) ==0:
                continue
            
            gain = info_gain(true_rows,false_rows,current_uncertainty)
            
            if gain >= best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question

class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)
        
        
class decision_node:
    
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch
    
def build_tree(rows):
    gain,question = find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    
    true_rows,false_rows = partition(rows,question)
    true_branch=build_tree(true_rows)
    false_branch=build_tree(false_rows)
    return decision_node(question,true_branch,false_branch)
def print_tree(node,spacing=" "):
    if isinstance(node,Leaf):
        print(spacing + "predict",node.predictions)
        return
    print(spacing + str(node.question))
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    print(spacing + '--> False:')
    print_tree(node.false_branch,spacing + " ")
    
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs={}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    
    #evaluate
    testing_data =[
        ['green',3,'apple'],
        ['yellow',4,'apple'],
        ['red',2,'grape'],
        ['red',1,'grape'],
        ['yellow',3,'lemon']
    ]
    for row in testing_data:
        print("actual: %s. predicted: %s" %
             (row[-1],print_leaf(classify(row,my_tree))))

 is diameter >= 3?
 --> True:
  is color == yellow?
  --> True:
   predict {'mango': 1, 'lemon': 1}
  --> False:
   predict {'mango': 1}
 --> False:
  predict {'grape': 2}
actual: apple. predicted: {'mango': '100%'}
actual: apple. predicted: {'mango': '50%', 'lemon': '50%'}
actual: grape. predicted: {'grape': '100%'}
actual: grape. predicted: {'grape': '100%'}
actual: lemon. predicted: {'mango': '50%', 'lemon': '50%'}
